# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.3

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 24
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(18331, 336)
(18331, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.59060403 -0.0455243  -0.27982732 ... -0.45454545  0.33333333
   0.33501825]
 [-0.36912752  0.0629156  -0.18014129 ... -0.27272727  0.25581395
   0.50126369]
 [-0.23489933 -0.13299233 -0.13854003 ... -0.09090909  0.11627907
   0.46222971]
 [-0.23489933 -0.39897698 -0.2299843  ... -0.27272727  0.31782946
   0.55293457]
 [-0.22147651 -0.39590793 -0.05926217 ... -0.81818182  0.34883721
   0.09491716]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Testing data (X, y)
[[-0.10738255 -0.49616368 -0.26844584 ... -0.27272727  0.03875969
   0.29458017]
 [-0.12751678 -0.18158568  0.02197802 ... -0.63636364  0.03875969
   0.0322943 ]
 [ 0.06711409 -0.05473146  0.13814757 ...  0.09090909  0.2248062
  

# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = nFeatures*tModel.data_handler.sequence_length


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')


tModel.epochs = 100

sess = tf.Session()

tModel.train_model(verbose=1)

Epoch 1/100
18331/18331 [==============================] - 1s 33us/step - loss: 8261.8681 - mean_squared_error: 8244.2128
Epoch 2/100
18331/18331 [==============================] - 0s 6us/step - loss: 5624.7958 - mean_squared_error: 5600.5249
Epoch 3/100
18331/18331 [==============================] - 0s 6us/step - loss: 2560.9277 - mean_squared_error: 2524.8375
Epoch 4/100
18331/18331 [==============================] - 0s 6us/step - loss: 1536.4842 - mean_squared_error: 1495.1546
Epoch 5/100
18331/18331 [==============================] - 0s 6us/step - loss: 1034.4464 - mean_squared_error: 992.1865
Epoch 6/100
18331/18331 [==============================] - 0s 6us/step - loss: 661.7560 - mean_squared_error: 617.8253
Epoch 7/100
18331/18331 [==============================] - 0s 6us/step - loss: 518.9438 - mean_squared_error: 473.8188
Epoch 8/100
18331/18331 [==============================] - 0s 6us/step - loss: 490.2484 - mean_squared_error: 444.5680
Epoch 9/100
18331/18331 [=============

18331/18331 [==============================] - 0s 6us/step - loss: 268.1028 - mean_squared_error: 231.6012
Epoch 70/100
18331/18331 [==============================] - 0s 6us/step - loss: 267.6706 - mean_squared_error: 231.3481
Epoch 71/100
18331/18331 [==============================] - 0s 7us/step - loss: 266.4829 - mean_squared_error: 230.3151
Epoch 72/100
18331/18331 [==============================] - 0s 7us/step - loss: 266.1004 - mean_squared_error: 230.1072
Epoch 73/100
18331/18331 [==============================] - 0s 6us/step - loss: 265.7485 - mean_squared_error: 229.8950
Epoch 74/100
18331/18331 [==============================] - 0s 6us/step - loss: 265.1076 - mean_squared_error: 229.4149
Epoch 75/100
18331/18331 [==============================] - 0s 6us/step - loss: 264.1906 - mean_squared_error: 228.5949
Epoch 76/100
18331/18331 [==============================] - 0s 6us/step - loss: 263.9921 - mean_squared_error: 228.5238
Epoch 77/100
18331/18331 [===========================

In [10]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, tf_session=sess)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 788us/step
[115. 128.  49.  80.  90. 110. 112.  90. 116.  97.  84. 101.  95. 102.
 112. 115.  56.  23.  94.   8.  84. 125. 134.  11. 114. 119. 108. 110.
  99. 108.   5.  52. 105.   7.  11.  24.  24.  54. 134.  25.  33.  20.
  64. 116.  85.  54. 115.  99.  13.  79. 120.  29.  34. 122. 118.  18.
 101.  40. 107. 118.  15.  49.  84.  22. 123.  13. 116.   4. 110.  96.
 112.  61. 119. 104. 111.   5.  33. 132.  93.  95.   4.   9. 124.  72.
 127. 107. 118. 119. 113.  27.  19.  20.  57.  60. 111. 106.  89.  86.
 113.  12.]
[112.  98.  69.  82.  91.  93.  91.  95. 111.  96.  97. 124.  95. 107.
  83.  84.  50.  28.  87.  16.  57. 111. 113.  20. 145. 119.  66.  97.
  90. 115.   8.  48. 106.   7.  11.  19.  21.  50. 142.  28.  18.  10.
  59. 109. 114.  47. 135.  92.  21.  79. 114.  29.  26.  97. 137.  15.
 103.  37. 114. 100.  21.  54.  72.  28. 128.  14.  77.   8. 121.  94.
 118.  50. 131. 126. 113.  10.  34. 107.  63.  90.   8.   9. 137.  58.
 118.  8